In [410]:
import numpy as np
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)


#bring in dipoles and energies
dipoles = np.load("LiH_r_1.4_6311g_fci_Dipoles_3.npy")
energies = np.load("LiH_r_1.4_6311g_fci_Energies_3.npy")
print(energies)
print(dipoles)


#lambda vectors 
lambda_vec = np.array([0.0, 0.0, 0.05])




omega = 0.12086 


m_n = 0 #num_photons


mu_n = 0 #electronic state


n_elec = 50

energies = energies[0:n_elec]



[-8.01219476 -7.899962   -7.88593596 -7.86104863 -7.86104863 -7.84851087
 -7.84851087 -7.80693549 -7.78459936 -7.76702524 -7.76342939 -7.75827098
 -7.75827098 -7.75123644 -7.75123644 -7.72171149 -7.68303148 -7.53167595
 -7.46183148 -7.44561152 -7.44561152 -7.44122102 -7.41700688 -7.41700688
 -7.41499491 -7.41499491 -7.40506676 -7.40302238 -7.39081209 -7.38221021
 -7.38221021 -7.37219855 -7.37219855 -7.36897222 -7.36897222 -7.35609042
 -7.35609042 -7.33714534 -7.33343267 -7.32130747 -7.31448363 -7.31448363
 -7.306799   -7.306799   -7.30583178 -7.30363511 -7.29983049 -7.29983049
 -7.29710106 -7.28736609]
[[[-6.58046359e-15  3.41157881e-14 -2.09867294e+00]
  [ 2.78190247e-15 -3.60685480e-14  5.86355257e-13]
  [-4.30257733e-15  4.18478564e-14 -9.00621585e-01]
  [ 3.67415785e-14 -1.19765363e-13 -6.68352408e-14]
  [-4.96295546e-13 -7.85195887e-14  1.49124188e-15]
  [-2.66638853e-01  1.45745802e+00  6.76181589e-14]
  [ 1.45745802e+00  2.66638853e-01  1.85958696e-15]
  [-2.19653815e-15  1.0165

In [411]:

def build_d_matrix(lambda_vec, dipoles):
    #return np.einsum('k, ijk -> ij', lambda_vec, dipoles[:mu_n, : mu_n, :])
    #return np.einsum('k, ijk -> ij', lambda_vec, dipoles)
    return np.einsum('k, ijk -> ij', lambda_vec, dipoles[:n_elec, : n_elec, :])
    #return np.tensordot(dipoles, lambda_vec, axes = 1)

def build_d_matrix_2(lambda_vec, dipoles, n_el):
        _d = np.zeros((n_el, n_el))
        for a in range(n_el):
            for b in range(n_el):
                _d[a, b] = np.dot(lambda_vec, dipoles[a, b, :])

        return _d




d_matrix = build_d_matrix(lambda_vec, dipoles)
d_matrix = build_d_matrix_2(lambda_vec, dipoles, n_elec)



print(d_matrix[0:4, 0:4])



def calc_first_order_energy_correction(d_matrix, omega, mu_n):

    E_n_1 = 0
    for gamma in range(0, len(energies)):
        E_n_1 += d_matrix[mu_n][gamma]*d_matrix[gamma][mu_n]

    E_n_1 = omega* E_n_1 

    return E_n_1                




def delta(i,j):
    if i ==j:
        return 1
    else:
        return 0

#unsimplified version of second order energy correction to check if the same answer is gotten
def calc_second_order_energy_correction_unsimplified(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):

            #for m_l in range(max(m_n-1,0), m_n+2):
            for m_l in range(max(0, m_n-1), m_n+2 ):

                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    sum_over_energy_states_gamma = 0
                    for gamma in range(0, len(energies)):
                        #numerator += d_matrix[gamma][mu_l]*d_matrix[mu_l][gamma]
                        sum_over_energy_states_gamma += (d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n] * delta(m_l, m_n))

                    
                    E_n_2 += (((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n + 1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega))

                    #print((((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n +1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega)))


    return E_n_2




def calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):
            for m_l in range(max(m_n-1,0), m_n+2):
                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    if m_l == m_n + 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega)

                        #print( d_matrix[mu_l][mu_n]  )


                    elif m_l  == m_n - 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n)) ** 2 )/(energies[mu_n] - energies[mu_l] + omega)

                        #print(( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega))

        
                    elif m_l  == m_n:

                        
                        numerator = 0
                        for gamma in range(0, len(energies)):
                            numerator += d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n]

                            #print("numerator: ", numerator)


                            #print("d_matrix[mu_l][gamma]: ",  d_matrix[mu_l][gamma] ," d_matrix[gamma][mu_n]: ", d_matrix[gamma][mu_n])
                

                        E_n_2 += (numerator**2) / ((energies[mu_n] - energies[mu_l]))

                        #print("denominator:" , ((energies[mu_n] - energies[mu_l])))

                        #print("contrib: ",(numerator**2) / ((energies[mu_n] - energies[mu_l])) )



    E_n_2 = E_n_2 * omega**2
    return E_n_2



def calc_energy_correction_to_first_order(mu_n, E_n_1):
    return energies[mu_n] + (omega * m_n) + E_n_1 

def calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 

def calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 + E_n_3



E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)


E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)



print("E_n_1: ", E_n_1)
print("E-n_2: ", E_n_2)


print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))

print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))

qed_fci_energies = np.load("LiH_r_1.4_6311g_qedfci_Energies.npy")
print("qed_fci_energy:", qed_fci_energies[mu_n])

[[-1.04933647e-01  2.93177629e-14 -4.50310793e-02 -3.34176204e-15]
 [ 2.93177629e-14  8.79357755e-02 -4.68065589e-14  1.04400320e-15]
 [-4.50310793e-02 -4.68065589e-14  1.04096884e-01  8.06267674e-15]
 [-3.34176204e-15  1.04400320e-15  8.06267674e-15  4.16640161e-03]]
E_n_1:  0.002139800021418779
E-n_2:  -0.0016045019835372299
uncorrected_energy: -8.012194758186697
energy corrected to second order:  -8.011659460148815
qed_fci_energy: -8.00913664849758


In [412]:
"""
print("LiH full ci energy value: ",energies[0])

lambda_vec = np.array([0.0, 0.0, 0.025])
for i in range(0, 25):

    d_matrix = build_d_matrix(lambda_vec, dipoles)
    

    print("lambda vector: ", lambda_vec)
    E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)


    E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)


    print("energy corrected to first order: " , calc_energy_correction_to_first_order(mu_n, E_n_1))
    print("energy to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))


    lambda_vec[2] += -0.001

"""


'\nprint("LiH full ci energy value: ",energies[0])\n\nlambda_vec = np.array([0.0, 0.0, 0.025])\nfor i in range(0, 25):\n\n    d_matrix = build_d_matrix(lambda_vec, dipoles)\n    \n\n    print("lambda vector: ", lambda_vec)\n    E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)\n\n\n    E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)\n\n\n    print("energy corrected to first order: " , calc_energy_correction_to_first_order(mu_n, E_n_1))\n    print("energy to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))\n\n\n    lambda_vec[2] += -0.001\n\n'

In [413]:
"""
print("LiH full ci energy value: ",energies[0])

total_corrections = []

lambda_vec = np.array([0.0, 0.0, 0.05])

for i in reversed(range(2, 50)):

    n_elec = i
    energies = energies[0:n_elec]

    d_matrix = build_d_matrix(lambda_vec, dipoles)

    E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)

    E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)

    print(E_n_1)
    print(E_n_2)

    print("energy corrected to first order: " , calc_energy_correction_to_first_order(mu_n, E_n_1))
    print("energy to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))


    total_corrections.append(E_n_1+E_n_2)

import matplotlib.pyplot as plt


total_corrections.reverse()
#plt.plot(range(2,50), np.e ** np.array(total_corrections))
plt.plot(range(2,50),np.array(total_corrections))

plt.show()
plt.close()



E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)

print(qed_fci_energies[0] - energies)
"""

'\nprint("LiH full ci energy value: ",energies[0])\n\ntotal_corrections = []\n\nlambda_vec = np.array([0.0, 0.0, 0.05])\n\nfor i in reversed(range(2, 50)):\n\n    n_elec = i\n    energies = energies[0:n_elec]\n\n    d_matrix = build_d_matrix(lambda_vec, dipoles)\n\n    E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)\n\n    E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)\n\n    print(E_n_1)\n    print(E_n_2)\n\n    print("energy corrected to first order: " , calc_energy_correction_to_first_order(mu_n, E_n_1))\n    print("energy to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))\n\n\n    total_corrections.append(E_n_1+E_n_2)\n\nimport matplotlib.pyplot as plt\n\n\ntotal_corrections.reverse()\n#plt.plot(range(2,50), np.e ** np.array(total_corrections))\nplt.plot(range(2,50),np.array(total_corrections))\n\nplt.show()\nplt.close()\n\n\n\nE_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, en

In [414]:
#coherent state transformation

#mu => (50,50,3)
#np.dot(mu[0,0,:], lambda_vec)
print(d_matrix[0:4, 0:4])
d_expectation_value = np.dot(dipoles[0,0,:], lambda_vec)
print(d_expectation_value)
def coherent_state_d_matrix():
    d_matrix = np.einsum('k, ijk -> ij', lambda_vec, dipoles[:n_elec, : n_elec, :])
    d_matrix = d_matrix - np.dot(d_expectation_value, np.eye(n_elec, n_elec))

    return d_matrix

d_matrix = coherent_state_d_matrix()
print(d_matrix[0:4, 0:4])

E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)


E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)



print("E_n_1: ", E_n_1)
print("E-n_2: ", E_n_2)


print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))

print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))

qed_fci_energies = np.load("LiH_r_1.4_6311g_qedfci_Energies.npy")
print("qed_fci_energy:", qed_fci_energies[mu_n])

[[-1.04933647e-01  2.93177629e-14 -4.50310793e-02 -3.34176204e-15]
 [ 2.93177629e-14  8.79357755e-02 -4.68065589e-14  1.04400320e-15]
 [-4.50310793e-02 -4.68065589e-14  1.04096884e-01  8.06267674e-15]
 [-3.34176204e-15  1.04400320e-15  8.06267674e-15  4.16640161e-03]]
-0.10493364689983152
[[ 0.00000000e+00  2.93177629e-14 -4.50310793e-02 -3.34176204e-15]
 [ 2.93177629e-14  1.92869422e-01 -4.68065589e-14  1.04400320e-15]
 [-4.50310793e-02 -4.68065589e-14  2.09030531e-01  8.06267674e-15]
 [-3.34176204e-15  1.04400320e-15  8.06267674e-15  1.09100049e-01]]
E_n_1:  0.0008090020707984952
E-n_2:  -0.0002621235685486818
uncorrected_energy: -8.012194758186697
energy corrected to second order:  -8.011647879684446
qed_fci_energy: -8.00913664849758
